In [1]:
def find_connected_components(sc, edges, n_dist = 5):
    
    def cc_find_root(group, found_new_pair):
        node, values = group
        
        #find the smalled vertex id in the current adjecency list 
        smallest = min(values  + [node])
        
        
        emit = []
        #if smallest is not the node 
        #emit the pairs with smallest for each node in the group
        if smallest < node:
            emit.append((node, smallest))
            for vi in values:
                if vi != smallest:
                    emit.append((vi, smallest))
                    found_new_pair.add(1)
        return emit
    
    
    while True:
        edges.persist()
        found_new_pair = sc.accumulator(0)
        print ("JOB_LOG", "  ITERATION : ",n_dist )
        
        #ccf map
        s1 = edges.flatMap(lambda x : [(x[0], x[1]), (x[1], x[0])])
        #s1.take(1)
        
        #ccf reduce
        s2 = s1.groupByKey().map(lambda (x,y): (x, list(y)))
        #print s2.take(1)
        
        s3 = s2.flatMap(lambda x : cc_find_root(x, found_new_pair))
        s3.cache()
        print "JOB_LOG","total relations count:", s3.count() #necessary to perform action else accumulator will not update
        
        
        #ccf deduce
        edges.unpersist()
        edges = s3.distinct() 
        
        print "JOB_LOG","New pairs found :",found_new_pair.value
        if found_new_pair.value == 0 or n_dist == 0:
            break
        n_dist -= 1
    
    return edges.map(lambda x: (x[1], x[0])).groupByKey().map(lambda (x,y) : (x, list(y)))

In [3]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession, SQLContext
ss = SparkSession.builder.config("spark.sql.hive.verifyPartitionPath",
                                     "false").enableHiveSupport().getOrCreate()
sc = ss.sparkContext
sqlContext =  SQLContext(sc)

edges = sc.parallelize([("a","b"),
                       ("b", "c"),
                       ("d", "e"),
                       ("d", "f"),
                       ("g", "b")
                       ])
res = find_connected_components(sc, edges)
res.take(100)
